In [5]:
%matplotlib inline
import math
import paddle
from paddle import nn
from paddle.nn import functional as F
import d2l

In [6]:
# 定义起始和结束标识符
START_TOKEN = '^'
END_TOKEN = '$'

# 读取原始文件
file_path = 'male.txt'  # 替换为你的文件路径
with open(file_path, 'r') as file:
    names = file.readlines()

# 清理每个名字，并添加起始和结束标识符
processed_names = [f"{START_TOKEN}{name.strip()}{END_TOKEN}" for name in names]

# 获取最长的名字长度
max_length = max(len(name) for name in processed_names)

# 创建字符到索引的映射
all_chars = set(''.join(processed_names))
char_to_index = {char: idx + 1 for idx, char in enumerate(all_chars)}
index_to_char = {idx: char for char, idx in char_to_index.items()}
print(char_to_index)
# 将字符编码并保存为新的.txt文件，用0进行填充以保持相同长度
processed_file_path = 'processed_names.txt'  # 保存处理后的文件路径
with open(processed_file_path, 'w') as processed_file:
    for name in processed_names:
        encoded_name = [char_to_index[char] for char in name]
        # 使用0填充使每个编码后的名字长度相同
        encoded_name += [0] * (max_length - len(encoded_name))
        encoded_name = ' '.join(map(str, encoded_name))  # 用空格分隔编码以便保存
        processed_file.write(encoded_name + '\n')

{'Z': 1, 'y': 2, 'j': 3, 'Q': 4, 'h': 5, 'l': 6, 'W': 7, 'I': 8, 'g': 9, 'q': 10, 'o': 11, 'i': 12, 'a': 13, 'P': 14, 'M': 15, 'n': 16, 'f': 17, '$': 18, '^': 19, 'e': 20, 'b': 21, 'X': 22, 'D': 23, 'w': 24, 'H': 25, 'C': 26, 'U': 27, 'v': 28, 'B': 29, 'F': 30, 'K': 31, 'O': 32, 'z': 33, 't': 34, 'p': 35, 'm': 36, "'": 37, 'E': 38, 'L': 39, 'k': 40, 'd': 41, 'G': 42, 'S': 43, 'T': 44, 'V': 45, 'J': 46, 'c': 47, 's': 48, ' ': 49, 'A': 50, 'r': 51, 'u': 52, 'x': 53, 'N': 54, 'Y': 55, 'R': 56, '-': 57}


In [7]:
import numpy as np
file_path = 'processed_names.txt'  # 修改为你的文件路径
with open(file_path, 'r') as file:
    encoded_names = file.readlines()

# 转换为列表，每个元素是一个编码后的名字的列表
encoded_names = [
    list(map(int, name.strip().split())) for name in encoded_names
]
encoded_names = np.array(encoded_names)
print(encoded_names)

[[19 50 13 ...  0  0  0]
 [19 50 13 ...  0  0  0]
 [19 50 21 ...  0  0  0]
 ...
 [19 31 13 ...  0  0  0]
 [19 42 12 ...  0  0  0]
 [19 42 20 ...  0  0  0]]


In [8]:
import paddle
from paddle.io import DataLoader
from sklearn.model_selection import train_test_split

# 这里是你读取已编码名字的代码，确保已正确读取到 encoded_names

# 划分数据集：80% 训练集，10% 验证集，10% 测试集
train_names, val_test_names = train_test_split(encoded_names, test_size=0.2, random_state=42)
val_names, test_names = train_test_split(val_test_names, test_size=0.5, random_state=42)

# 创建数据加载器
batch_size = 32  # 设置 mini-batch 大小


# 创建 DataLoader
train_loader = DataLoader(
    dataset=train_names,
    batch_size=batch_size,
    shuffle=True,

)

val_loader = DataLoader(
    dataset=val_names,
    batch_size=batch_size,
    shuffle=False,

)

test_loader = DataLoader(
    dataset=test_names,
    batch_size=batch_size,
    shuffle=False,
)

# 打印输出一个mini-batch的数据
for idx, item in enumerate(train_loader()):
    X = item[:,:-1]
    Y = item[:,1:]
    # Y = F.one_hot(Y,len(char_to_index)+1)
    # Y = paddle.reshape(Y, shape=[-1, 58])
    print(X)
    print(Y)
    break

Tensor(shape=[32, 16], dtype=int32, place=Place(gpu:0), stop_gradient=True,
       [[19, 23, 13, 51, 41, 13, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 44, 20, 41, 12, 20, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 31, 13, 34, 12, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 43, 13, 2 , 20, 51, 48, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 50, 52, 21, 51, 20, 2 , 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 23, 11, 51, 12, 13, 16, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 46, 20, 21, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 38, 51, 51, 11, 6 , 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 45, 13, 6 , 41, 20, 36, 13, 51, 18, 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 46, 13, 16, 20, 47, 33, 40, 13, 18, 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 50, 16, 16, 13, 6 , 12, 48, 20, 18, 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 32, 51, 12, 13, 16, 13, 18, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
        [19, 39, 2 , 16, 20, 6 , 6 ,